# Video subtitles generation using Whisper and OpenVINO

Whisper is a general-purpose speech recognition model. It is a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification. In this demo we will use its capabilities for generation subtitles to video on several languages.

Notebook contains following steps:
1. Convert model to ONNX format.
2. Convert model to IR using OpenVINO Model Optimizer tool.
3. Run Whisper pipeline with OpenVINO models.

## Prerequisites

clone and install model repository

In [1]:
!git clone https://github.com/openai/whisper.git
%cd whisper
!python setup.py develop
%cd ..

c:\Users\eaidova\repos\openvino_notebooks\notebooks\225-whisper-subtitels-generation\whisper


fatal: destination path 'whisper' already exists and is not an empty directory.


c:\Users\eaidova\repos\openvino_notebooks\notebooks\225-whisper-subtitels-generation


python: can't open file 'c:\Users\eaidova\repos\openvino_notebooks\notebooks\225-whisper-subtitels-generation\whisper\setup.py': [Errno 2] No such file or directory


## Instantiate model

In [2]:
import whisper

model = whisper.load_model("medium")
model.eval()

c:\Users\eaidova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\eaidova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\eaidova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1024, 1024, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0): ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1024, out_features=1024, bias=True)
          (key): Linear(in_features=1024, out_features=1024, bias=False)
          (value): Linear(in_features=1024, out_features=1024, bias=True)
          (out): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (attn_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1024, out_features=4096, bias=True)
          (1): GELU(approximate=none)
          (2): Linear(in_features=4096, out_features=1024, bias=True)
        )
        (mlp_ln): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      )
      (1): ResidualAttentionBlock(
       

## Convert model to ONNX

Model consists of 2 parts:
* Encoder
* Decoder

### Whisper Encoder to ONNX

In [3]:
import torch
mel = torch.zeros((1, 80, 3000))

torch.onnx.export(model.encoder, mel, 'whisper_encoder.onnx', input_names=['mel'], output_names=['audio_features'])

c:\Users\eaidova\Anaconda3\lib\site-packages\whisper\model.py:152: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert x.shape[1:] == self.positional_embedding.shape, "incorrect audio shape"
c:\Users\eaidova\Anaconda3\lib\site-packages\whisper\model.py:90: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  scale = (n_state // self.n_head) ** -0.25


### Whisper decoder to ONNX

In [4]:
import torch
from typing import Optional, Union, List
from functools import partial

positional_embeddings_size = model.decoder.positional_embedding.shape[0]

def save_to_cache(cache, module, output):
    if module not in cache or output.shape[1] > positional_embeddings_size:
        cache[module] = output  # save as-is, for the first token or cross attention
    else:
        cache[module] = torch.cat([cache[module], output], dim=1).detach()
    return cache[module]
  
def attention_forward(
        attention_module,
        x: torch.Tensor,
        xa: Optional[torch.Tensor] = None,
        mask: Optional[torch.Tensor] = None,
        kv_cache: Optional[dict] = None,
        idx: int = 0
):
    q = attention_module.query(x)

    if kv_cache is None or xa is None:
        # hooks, if installed (i.e. kv_cache is not None), will prepend the cached kv tensors;
        # otherwise, perform key/value projections for self- or cross-attention as usual.
        k = attention_module.key(x if xa is None else xa)
        v = attention_module.value(x if xa is None else xa)
        if kv_cache is not None:
            k = save_to_cache(kv_cache, f'k_{idx}', k)
            v = save_to_cache(kv_cache, f'v_{idx}', v)
    else:
        # for cross-attention, calculate keys and values once and reuse in subsequent calls.
        k = kv_cache.get(f'k_{idx}', save_to_cache(kv_cache, f'k_{idx}', attention_module.key(xa)))
        v = kv_cache.get(f'v_{idx}', save_to_cache(kv_cache, f'v_{idx}', attention_module.value(xa)))

    wv = attention_module.qkv_attention(q, k, v, mask)
    return attention_module.out(wv), kv_cache


def block_forward(
        residual_block,
        x: torch.Tensor,
        xa: Optional[torch.Tensor] = None,
        mask: Optional[torch.Tensor] = None,
        kv_cache: Optional[dict] = None,
        idx:int = 0
    ):
        x0, kv_cache = residual_block.attn(residual_block.attn_ln(x), mask=mask, kv_cache=kv_cache, idx=f'{idx}a')
        x = x + x0
        if residual_block.cross_attn:
            x1, kv_cache = residual_block.cross_attn(residual_block.cross_attn_ln(x), xa, kv_cache=kv_cache, idx=f'{idx}c')
            x = x + x1
        x = x + residual_block.mlp(residual_block.mlp_ln(x))
        return x, kv_cache

for idx, block in enumerate(model.decoder.blocks):
    block.forward = partial(block_forward, block, idx=idx)
    block.attn.forward = partial(attention_forward, block.attn)
    if block.cross_attn:
        block.cross_attn.forward = partial(attention_forward, block.cross_attn)


def decoder_forward(decoder, x: torch.Tensor, xa: torch.Tensor, kv_cache: Optional[dict] = None):
    """
    x : torch.LongTensor, shape = (batch_size, <= n_ctx) the text tokens
    xa : torch.Tensor, shape = (batch_size, n_mels, n_audio_ctx)
        the encoded audio features to be attended on
    """
    offset = next(iter(kv_cache.values())).shape[1] if kv_cache else 0
    x = decoder.token_embedding(x) + decoder.positional_embedding[offset : offset + x.shape[-1]]
    x = x.to(xa.dtype)

    for block in decoder.blocks:
        x, kv_cache = block(x, xa, mask=decoder.mask, kv_cache=kv_cache)

    x = decoder.ln(x)
    logits = (x @ torch.transpose(decoder.token_embedding.weight.to(x.dtype), 1, 0)).float()

    return logits, kv_cache

model.decoder.forward = partial(decoder_forward, model.decoder)


In [5]:
tokens = torch.ones((5, 3), dtype=torch.int64)
audio_features = torch.ones((1, 1500, 1024))

logits, kv_cache = model.decoder(tokens, audio_features, kv_cache={})
kv_cache = {k: v for k, v in kv_cache.items()}
tokens = torch.ones((5, 1), dtype=torch.int64)

In [6]:
outputs = [f'out_{k}' for k in kv_cache.keys()]
inputs = [f'in_{k}' for k in kv_cache.keys()]
dynamic_axes = {'tokens': {0: 'beam_size', 1: 'seq_len'}, 'audio_features': {0: 'beam_size'}, 'logits': {0: 'beam_size', 1: 'seq_len'}}
dynamic_outs = {o: {0: 'beam_size', 1: 'prev_seq_len'} for o in outputs}
dynamic_inp = {i: {0: 'beam_size', 1: 'prev_seq_len'}  for i in inputs}
dynamic_axes.update(dynamic_outs)
dynamic_axes.update(dynamic_inp)
torch.onnx.export(
    model.decoder, {'x': tokens, 'xa': audio_features, 'kv_cache': kv_cache},
'whisper_decoder.onnx',
input_names=['tokens', 'audio_features'] + inputs,
output_names=['logits'] + outputs,
dynamic_axes=dynamic_axes
)

<ipython-input-4-1209e68a6a67>:8: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if module not in cache or output.shape[1] > positional_embeddings_size:


In [7]:
class OpenVINOAudioEncoder(torch.nn.Module):
    def __init__(self, core, model_path, device='CPU'):
        super().__init__()
        self.model = core.read_model(model_path)
        self.compiled_model = core.compile_model(self.model, device)
        self.output_blob = self.compiled_model.output(0)

    def forward(self, mel:torch.Tensor):
        return torch.from_numpy(self.compiled_model(mel)[self.output_blob])

In [8]:
class OpenVINOTextDecoder(torch.nn.Module):
    def __init__(self, core, model_path, device='CPU'):
        super().__init__()
        self._core = core
        self.model = core.read_model(model_path)
        self._input_names = [inp.any_name for inp in self.model.inputs]
        self.compiled_model = core.compile_model(self.model, device)
        self.device = device
    
    def init_past_inputs(self, feed_dict):
        beam_size = feed_dict['tokens'].shape[0]
        audio_len = feed_dict['audio_features'].shape[-1]
        previous_seq_len = 0
        for name in self._input_names:
            if name in ['tokens', 'audio_features']:
                continue
            feed_dict[name] = np.zeros((beam_size, previous_seq_len, audio_len), dtype=np.float32)
        return feed_dict

    def preprocess_kv_cache_inputs(self, feed_dict, kv_cache):
        if not kv_cache:
            return self.init_past_inputs(feed_dict)
        for k, v in kv_cache.items():
            new_k = f'in_{k}'
            if new_k in self._input_names:
                feed_dict[new_k] = v
        return feed_dict

    def postprocess_outputs(self, outputs):
        logits = None
        kv_cache = {}
        for output_t, out in outputs.items():
            if 'logits' in output_t.get_names():
                logits = torch.from_numpy(out)
            else:
                tensor_name = output_t.any_name
                kv_cache[tensor_name.replace('out_', '')] = torch.from_numpy(out)
        return logits, kv_cache

    def forward(self, x:torch.Tensor, xa:torch.Tensor, kv_cache: Optional[dict]=None):
        feed_dict = {'tokens': x, 'audio_features': xa}
        feed_dict = (self.preprocess_kv_cache_inputs(feed_dict, kv_cache))
        res = self.compiled_model(feed_dict)
        return self.postprocess_outputs(res)

In [9]:
from whisper.decoding import DecodingTask, Inference, DecodingOptions, DecodingResult


class OpenVINOInference(Inference):
    def __init__(self, model: "Whisper", initial_token_length: int):
        self.model: "Whisper" = model
        self.initial_token_length = initial_token_length
        self.kv_cache = {}
    
    def logits(self, tokens: torch.Tensor, audio_features: torch.Tensor) -> torch.Tensor:
        if tokens.shape[-1] > self.initial_token_length:
            # only need to use the last token except in the first forward pass
            tokens = tokens[:, -1:]
        logits, self.kv_cache = self.model.decoder(tokens, audio_features, kv_cache=self.kv_cache)
        return logits

    def cleanup_caching(self):
        self.kv_cache = {}

    def rearrange_kv_cache(self, source_indices):
        for module, tensor in self.kv_cache.items():
            # update the key/value cache to contain the selected sequences
            self.kv_cache[module] = tensor[source_indices]


class OpenVINODecodingTask(DecodingTask):
    def __init__(self, model: "Whisper", options: DecodingOptions):
        super().__init__(model, options)
        self.inference = OpenVINOInference(model, len(self.initial_tokens))

@torch.no_grad()
def decode(model: "Whisper", mel: torch.Tensor, options: DecodingOptions = DecodingOptions()) -> Union[DecodingResult, List[DecodingResult]]:
    """
    Performs decoding of 30-second audio segment(s), provided as Mel spectrogram(s).

    Parameters
    ----------
    model: Whisper
        the Whisper model instance

    mel: torch.Tensor, shape = (80, 3000) or (*, 80, 3000)
        A tensor containing the Mel spectrogram(s)

    options: DecodingOptions
        A dataclass that contains all necessary options for decoding 30-second segments

    Returns
    -------
    result: Union[DecodingResult, List[DecodingResult]]
        The result(s) of decoding contained in `DecodingResult` dataclass instance(s)
    """
    single = mel.ndim == 2
    if single:
        mel = mel.unsqueeze(0)

    result = OpenVINODecodingTask(model, options).run(mel)
    
    if single:
        result = result[0]

    return result


In [13]:
from openvino.tools import mo
from openvino.runtime import serialize
encoder_model = mo.convert(input_model='whisper_encoder.onnx', data_type='FP16')
serialize(encoder_model, 'whisper_encoder.xml')

input_shapes = 'tokens[1..5 1..224],audio_features[1..5 1500 1024]'
for k, v in kv_cache.items():
    if k.endswith('a'):
        input_shapes += f',in_{k}[1..5 0..224 1024]' 
decoder_model = mo.convert(input_model='whisper_decoder.onnx', data_type='FP16', input=input_shapes)
serialize(decoder_model, 'whisper_decoder.xml')


In [14]:
del model.decoder
del model.encoder

In [15]:
from openvino.runtime import Core
from collections import namedtuple

Parameter = namedtuple('Parameter', ['device'])

core = Core()

model.encoder = OpenVINOAudioEncoder(core, 'whisper_encoder.xml')
model.decoder = OpenVINOTextDecoder(core, 'whisper_decoder.xml')
model.decode = partial(decode, model)

def parameters():
    return iter([Parameter(torch.device('cpu'))])

model.parameters = parameters

def logits(model, tokens: torch.Tensor, audio_features: torch.Tensor):
    return model.decoder(tokens, audio_features, None)[0]

model.logits = partial(logits, model)


In [16]:
import io
from moviepy.editor import VideoFileClip
import numpy as np
from scipy.io import wavfile

def resample(audio, src_sample_rate, dst_sample_rate):
    if src_sample_rate == dst_sample_rate:
        return audio
    duration = audio.shape[0] / src_sample_rate
    resampled_data = np.zeros(shape=(int(duration * dst_sample_rate)), dtype=np.float32)
    x_old = np.linspace(0, duration, audio.shape[0], dtype=np.float32)
    x_new = np.linspace(0, duration, resampled_data.shape[0], dtype=np.float32)
    resampled_audio = np.interp(x_new, x_old, audio)
    return resampled_audio.astype(np.float32)

def audio_to_float(audio):
    return audio.astype(np.float32) / np.iinfo(audio.dtype).max


def get_audio(video_file):
    input_video = VideoFileClip(str(video_file))
    input_video.audio.write_audiofile(video_file.stem + '.wav')
    input_audio_file = video_file.stem + '.wav'
    sample_rate, audio = wavfile.read(io.BytesIO(open(input_audio_file, 'rb').read()))
    audio = audio_to_float(audio)
    if audio.ndim == 2:
        audio = audio.mean(axis=1)
    resampled_audio = resample(audio, sample_rate, 16000)
    return resampled_audio

In [17]:
from pathlib import Path
video_file = Path("../../../whisper/Nono_ch_15s.mp4")
audio = get_audio(video_file)

MoviePy - Writing audio in Nono_ch_15s.wav


MoviePy - Done.


In [18]:
transcription = model.transcribe(audio, beam_size=5, best_of=5, task='translate')

Detected language: chinese


In [23]:
def format_timestamp(seconds: float):
    assert seconds >= 0, "non-negative timestamp expected"
    milliseconds = round(seconds * 1000.0)

    hours = milliseconds // 3_600_000
    milliseconds -= hours * 3_600_000

    minutes = milliseconds // 60_000
    milliseconds -= minutes * 60_000

    seconds = milliseconds // 1_000
    milliseconds -= seconds * 1_000

    return (f"{hours}:" if hours > 0 else "00:") + f"{minutes:02d}:{seconds:02d},{milliseconds:03d}"

def prepare_srt_bilingual(transcription, translation):
    segment_lines = []
    for segment1, segment2 in zip(transcription['segments'], translation['segments']):
        segment_lines.append(str(segment1['id'] + 1) +'\n')
        time_start = format_timestamp(segment1['start'])
        time_end = format_timestamp(segment1['end'])
        time_str = f'{time_start} --> {time_end}\n'
        segment_lines.append(time_str)
        segment_lines.append(segment1['text'] + '\n' + segment2['text'] + '\n\n')
    return segment_lines


def prepare_srt(transcription):
    segment_lines = []
    for segment in transcription['segments']:
        segment_lines.append(str(segment['id'] + 1) +'\n')
        time_start = format_timestamp(segment['start'])
        time_end = format_timestamp(segment['end'])
        time_str = f'{time_start} --> {time_end}\n'
        segment_lines.append(time_str)
        segment_lines.append(segment['text'] + '\n\n')
    return segment_lines

In [24]:
srt_lines = prepare_srt(transcription)

In [26]:
print(''.join(srt_lines))

1
00:00:00,000 --> 00:00:02,600
 The world of AI is developing rapidly.

2
00:00:02,600 --> 00:00:04,760
 From the technology little white to a technical big cow,

3
00:00:04,760 --> 00:00:07,360
 Complete the gorgeous transformation of professional life.

4
00:00:07,360 --> 00:00:35,360
 Unlock the Intel AI Developer Certification Certified.




In [1]:
from openvino.runtime import Core
core = Core()
encoder_model = core.read_model('whisper_encoder.xml')

c:\Users\eaidova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\eaidova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\eaidova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import torchaudio
from pathlib import Path
import torch
import whisper

Path('librispeech').mkdir(parents=True, exist_ok=True)
class LibriSpeech(torch.utils.data.Dataset):
    """
    A simple class to wrap LibriSpeech and trim/pad the audio to 30 seconds.
    It will drop the last few seconds of a very small portion of the utterances.
    """
    def __init__(self, split="test-clean", device='cpu'):
        self.dataset = torchaudio.datasets.LIBRISPEECH(
            root='librispeech',
            url=split,
            download=True,
        )
        self.device = device

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        audio, sample_rate, text, _, _, _ = self.dataset[item]
        assert sample_rate == 16000
        audio = whisper.pad_or_trim(audio.flatten()).to(self.device)
        mel = whisper.log_mel_spectrogram(audio)
        
        return (mel, text)

In [3]:
dataset = LibriSpeech("test-clean")
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

In [4]:
# Define the transformation method. This method should
# take a data item from the data source and transform it
# into the model expected input.
def transform_fn(data_item):
    mel, _ = data_item
    return mel.numpy()

In [5]:
from nncf import ptq
from openvino.runtime import serialize
# Wrap framework-specific data source into `NNCFDataLoader` object.
validation_dataset = ptq.create_dataloader(loader, transform_fn)

quantized_encoder_model = ptq.quantize(encoder_model, validation_dataset, model_type='transformer')
serialize(quantized_encoder_model, 'whisper_encoder_int8.xml')